In [10]:
import os,ogr

In [2]:
import pyspark

In [3]:
sc = pyspark.SparkContext()

In [4]:
eqcsv = os.getcwd()+"\\data\\eq2013.csv"

In [7]:
rdd = sc.textFile(eqcsv)

In [8]:
rdd.count()

8205

In [9]:
rdd.take(10)

['date,time,x,y,z,lv,M,v1,v2,v3,v4',
 '2013/5/31,23:41:56,-113.408,37.175,6.6,2.5,ML2.5,SLC,,UTAH,',
 '2013/5/31,23:09:05,-113.411,37.178,6,2.5,ML2.5,SLC,,UTAH,',
 '2013/5/31,22:45:34,-113.413,37.172,4,2.9,ML2.9,SLC,,UTAH,',
 '2013/5/31,22:34:26,-113.414,37.174,3.2,2.8,ML2.8,SLC,,UTAH,',
 '2013/5/31,22:34:02,-178.08,51.127,26,3.1,ML3.1,AEIC,,ANDREANOF ISLANDS, ALEUTIAN IS.',
 '2013/5/31,22:21:56,143.032,21.648,322.5,4.4,,,,MARIANA ISLANDS REGION,',
 '2013/5/31,21:17:52,-64.754,19.103,36.9,2.9,MD2.9,RSPR,,VIRGIN ISLANDS,',
 '2013/5/31,20:17:11,161.56,-10.28,58.8,4.7,,,,SOLOMON ISLANDS,',
 '2013/5/31,19:51:33,-121.068,40.148,6.6,2.2,MD2.2,NC,,NORTHERN CALIFORNIA,']

## 编写一个过滤的方法。
### 原理是用xy两个字段来构造一个点要素，然后用gdal的IsValid方法来验证，如果能用，就表示是这一行是一个合法的要素描述信息

In [ ]:
def isPoint(line):
    pntline = line.split(",")
    try:
        wkt = "POINT({0} {1})".format(float(pntline[2]),float(pntline[2]))
        geom = ogr.CreateGeometryFromWkt(wkt)
        return geom.IsValid()
    except:
        return False

In [12]:
eqrdd = rdd.filter(lambda line : isPoint(line))

## 执行过滤之后，发现减少了一行

In [13]:
eqrdd.count()

8204

In [14]:
eqrdd.take(10)

['2013/5/31,23:41:56,-113.408,37.175,6.6,2.5,ML2.5,SLC,,UTAH,',
 '2013/5/31,23:09:05,-113.411,37.178,6,2.5,ML2.5,SLC,,UTAH,',
 '2013/5/31,22:45:34,-113.413,37.172,4,2.9,ML2.9,SLC,,UTAH,',
 '2013/5/31,22:34:26,-113.414,37.174,3.2,2.8,ML2.8,SLC,,UTAH,',
 '2013/5/31,22:34:02,-178.08,51.127,26,3.1,ML3.1,AEIC,,ANDREANOF ISLANDS, ALEUTIAN IS.',
 '2013/5/31,22:21:56,143.032,21.648,322.5,4.4,,,,MARIANA ISLANDS REGION,',
 '2013/5/31,21:17:52,-64.754,19.103,36.9,2.9,MD2.9,RSPR,,VIRGIN ISLANDS,',
 '2013/5/31,20:17:11,161.56,-10.28,58.8,4.7,,,,SOLOMON ISLANDS,',
 '2013/5/31,19:51:33,-121.068,40.148,6.6,2.2,MD2.2,NC,,NORTHERN CALIFORNIA,',
 '2013/5/31,19:29:31,-67.355,18.185,12,2.7,MD2.7,RSPR,,MONA PASSAGE,']